In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
# Initialize a BigQuery client
client = bigquery.Client()

# Specify your query


query20 = """
WITH Medias AS (
  SELECT 'aposentados_civil' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
  FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_aposentados_remuneracao`
  UNION ALL
  SELECT 'pensionista_civil' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
  FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_pensionista_remuneracao`
  UNION ALL
  SELECT 'militar_reserva_reforma' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
  FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_reserva_reforma_remuneracao`
  UNION ALL
  SELECT 'pensionista_militar' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
  FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_pensionistas_remuneracao`
),

Medianas AS (
  SELECT 'aposentados_civil' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_siape_aposentados_remuneracao`
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))

  UNION ALL

  SELECT 'pensionista_civil' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_siape_pensionista_remuneracao`
    
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))
UNION ALL

  SELECT 'militar_reserva_reforma' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_militares_reserva_reforma_remuneracao`
    
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))
  UNION ALL

  SELECT 'pensionista_militar' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_militares_pensionistas_remuneracao`
    
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))


)

SELECT 
  M.detalhamento,
  M.media,
  MD.mediana
FROM Medias M
LEFT JOIN Medianas MD
ON M.detalhamento = MD.detalhamento
"""

# Execute the query
query_job = client.query(query20)

In [ ]:
df20 = client.query(query20).to_dataframe()

In [ ]:
df20['ano']=2020
df20 = df20[df20['mediana'].notna()]
df20

In [ ]:
### Verificar distribuição das remunerações: 

df['REMUNERACAO_BASICA_BRUTA_EM_REAIS'].describe()

In [ ]:
anos = [2020,2021,2022,2023]

In [ ]:
resultados=[]

In [ ]:
for ano in anos: 
  query = f"""
  WITH Medias AS (
    SELECT 'aposentados_civil' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_aposentados_remuneracao`
   UNION ALL
   SELECT 'pensionista_civil' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_pensionista_remuneracao`
   UNION ALL
   SELECT 'militar_reserva_reforma' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
   FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_reserva_reforma_remuneracao`
   UNION ALL
   SELECT 'pensionista_militar' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
   FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_pensionistas_remuneracao`
  ),

  Medianas AS (
  SELECT 'aposentados_civil' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_aposentados_remuneracao`
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))

  UNION ALL

  SELECT 'pensionista_civil' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_pensionista_remuneracao`
    
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))
  UNION ALL

  SELECT 'militar_reserva_reforma' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_reserva_reforma_remuneracao`
    
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))
  UNION ALL

  SELECT 'pensionista_militar' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS, 
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_pensionistas_remuneracao`
    
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))


  )

  SELECT 
    M.detalhamento,
  M.media,
  MD.mediana
FROM Medias M
LEFT JOIN Medianas MD
ON M.detalhamento = MD.detalhamento
"""

# Execute the query
  query_job = client.query(query)
  df_name = f"df{ano}"
  exec(f"{df_name} = query_job.to_dataframe()")

In [ ]:
anos2 = [2021,2022,2023]
df2020['ano'] = 2020 # Adiciona a coluna 'ano'
df2020= df2020[df2020['mediana'].notna()] 

In [ ]:
for ano in anos2: 
   # Cria o nome da variável do DataFrame com base no ano
    df_name = f"df{ano}"
    
    # Recupera o DataFrame correspondente ao ano
    df = globals().get(df_name)  # Usa globals() para obter a variável global com o nome df{ano}
    
    if df is not None:  # Verifica se o DataFrame existe
        df['ano'] = ano  # Adiciona a coluna 'ano'
        df = df[df['mediana'].notna()]  # Filtra linhas onde 'mediana' não é NaN
        
        # Concatena o DataFrame atual ao df2020
        df2020 = pd.concat([df2020, df], ignore_index=True)
   

In [ ]:
df=df2020
df

In [ ]:
## Fazer a correção pelo ipca: 

ipca_query = """
SELECT * FROM `repositoriodedadosgpsp.Datalake.ipca`
"""
ipca = client.query(ipca_query).to_dataframe()

In [ ]:
df1=df.merge(ipca[['ano','fator_correcao']], how='left', on='ano')

In [ ]:
df1['media_corrigido']= df1['media']*df1['fator_correcao']
df1['mediana_corrigido']= df1['mediana']*df1['fator_correcao']

In [ ]:
df1.columns

In [ ]:
df1=df1[['ano','detalhamento', 'media', 'mediana','media_corrigido', 'mediana_corrigido','fator_correcao']]

In [ ]:
df1.info()